In [28]:
from pathlib import Path
import os

# Establece la raíz del proyecto manualmente
project_root = Path("F:/JCMDataCenter/Cursos/Evolve Academy/Data Scientist IA/Futpeak") # sobremesa
#project_root = Path("C:/Users/juanm/Desktop/FUTPEAK/Futpeak") # portátil

# Cambia el directorio de trabajo actual a esa raíz
os.chdir(project_root)

print("📁 Directorio de trabajo actual:", Path.cwd())

📁 Directorio de trabajo actual: F:\JCMDataCenter\Cursos\Evolve Academy\Data Scientist IA\Futpeak


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.neighbors import NearestNeighbors
import os


In [30]:
# Cargar modelo entrenado
model = CatBoostClassifier()
model.load_model("model/catboost_multiclase_futpeak.cbm")


In [42]:
import pandas as pd
import numpy as np

# === Load historical data
main_logs = pd.read_csv("data/processed/cleaned_matchlogs.csv")
main_meta = pd.read_csv("data/processed/cleaned_metadata.csv")
main_df = main_logs.merge(main_meta, on="Player_ID")

# Preferimos la Position de metadata (más estable)
if "Position_y" in main_df.columns:
    main_df.rename(columns={"Position_y": "Position"}, inplace=True)
if "Position_x" in main_df.columns:
    main_df.drop(columns=["Position_x"], inplace=True)

# === Load Lamine's data
future_df = pd.read_csv("data/processed/future_stars_cleaned_matchlogs.csv")
future_meta = pd.read_csv("data/processed/future_stars_cleaned_metadata.csv")
future_df = future_df.merge(future_meta, on="Player_ID", how="left")

if "Position_y" in future_df.columns:
    future_df.rename(columns={"Position_y": "Position"}, inplace=True)
if "Position_x" in future_df.columns:
    future_df.drop(columns=["Position_x"], inplace=True)

# === Rating columns to process
rating_cols = ['Goals', 'Assists', 'Shots', 'Shots_on_target', 'Yellow_cards', 'Red_cards', 'Minutes']

# === Ensure numeric and compute rating
def calculate_rating(row):
    score = (
        row['Goals'] * 5 +
        row['Assists'] * 4 +
        row['Shots_on_target'] * 0.5 +
        (row['Shots'] - row['Shots_on_target']) * 0.1 -
        row['Yellow_cards'] * 1 -
        row['Red_cards'] * 2
    )
    return score / (row['Minutes'] / 90) if row['Minutes'] > 0 else 0

for df in [main_df, future_df]:
    for col in rating_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    df["rating_per_90"] = df.apply(calculate_rating, axis=1)

print("✅ Matchlogs loaded and rating_per_90 calculated.")



✅ Matchlogs loaded and rating_per_90 calculated.


In [43]:
# Convert date columns to datetime with format specified
for df in [main_df, future_df]:
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")
    df["Birth_date"] = pd.to_datetime(df["Birth_date"], format="%Y-%m-%d", errors="coerce")
    df["Age"] = (df["Date"] - df["Birth_date"]).dt.days / 365.25

print("📆 Age calculated from birth_date and match date.")


📆 Age calculated from birth_date and match date.


In [44]:
print(main_df.columns[main_df.columns.duplicated()])



Index([], dtype='object')


In [45]:
# Define position groups
POSITION_GROUPS = {
    "GOALKEEPER": ["GK"],
    "DEFENSIVE": ["CB", "LB", "RB", "FB", "LWB", "RWB", "SW", "DF"],
    "MIDFIELD": ["CM", "DM", "MF", "AM"],
    "ATTACKING": ["CF", "ST", "FW", "LW", "RW", "WF", "IF", "OL", "OR"]
}
target_positions = POSITION_GROUPS["MIDFIELD"] + POSITION_GROUPS["ATTACKING"]

# Function to detect if a position is offensive or creative
def is_target_position(pos):
    if isinstance(pos, str):
        return any(p in pos.split('-') for p in target_positions)
    return False

# Apply filtering
main_df["Is_Target"] = main_df["Position"].apply(is_target_position)
future_df["Is_Target"] = future_df["Position"].apply(is_target_position)

# Keep only relevant players
main_df = main_df[main_df["Is_Target"]].copy()
future_df = future_df[future_df["Is_Target"]].copy()

print("🎯 Filtered to attacking/midfield positions.")


🎯 Filtered to attacking/midfield positions.


In [46]:
# Set Lamine's Player_ID
lamine_id = "82ec26c1"

# Ensure Lamine is present
if lamine_id not in future_df["Player_ID"].values:
    raise ValueError("❌ Lamine Yamal not found in future_df!")

# Get latest available match
lamine_latest = future_df[future_df["Player_ID"] == lamine_id].sort_values("Date").iloc[-1]

print("🧩 Selected Lamine's latest match entry.")



🧩 Selected Lamine's latest match entry.
